In [58]:
import pathlib
from datetime import datetime
import time

import torch
from torch import cuda

import glob
import json
import os
import shutil
import sys
import ast
import random
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
import skimage.io as io
from shapely import Polygon
from matplotlib import pyplot as plt
from PIL import Image
from torch import cuda
import supervision as sv
from ultralytics import YOLO
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.auto import tqdm, trange
from importlib import reload

# Clone FTCNN repo here: https://www.github.com/joeletho/FTCNN.git

# Parent directory of cloned repo
sys.path.append(CHANGE_ME)

import ftcnn

from ftcnn import ftcnn as ft

In [59]:
print(sys.version)

3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]


In [60]:
print(reload(ftcnn))

<module 'ftcnn' from '/home/joel/Dev/python/ftcnn/__init__.py'>


In [61]:
has_gpu = cuda.is_available()

device = torch.device('cuda' if has_gpu else 'cpu')
print(device)
if has_gpu:
    print(cuda.get_device_name(0))

cuda
NVIDIA GeForce RTX 3050 Ti Laptop GPU


#### Example directory structure:
```
Root
  ├── datasets
  ├── FTCNN_YOLO
  ├── models
  ├── NDVI
  ├── QGIS
  ├── Readme.txt
  ├── runs
  ├── Shapefiles
```

In [62]:
path_map = {'ROOT': Path(CHANGE_ME_TO_YOUR_LOCAL_DIR)}
path_map['NDVI'] = path_map['ROOT'] / 'NDVI' / 'NDVI Difference Rasters'
path_map['SHAPE_FILES'] = path_map['ROOT'] / 'Shapefiles'

In [63]:
CHIP_SIZE = 128
YEARS=(2022, 2023)
PRETRAINED=True
SPLIT_MODE='all'
path_map['MODEL_NAME'] = f"yolo_ftcnn_img-years={YEARS[0]}-{YEARS[1]}_geom-years=ALL_chipsz={CHIP_SIZE if CHIP_SIZE is not None else 'Default'}_{SPLIT_MODE}"

path_map['PROJECT_NAME'] = 'FTCNN_YOLO'

In [64]:
path_map['FTCNN'] = path_map['ROOT'] / path_map["PROJECT_NAME"]

path_map['FTCNN_DS'] = path_map['ROOT'] / 'datasets'
path_map['FTCNN_DS_META'] = path_map['FTCNN_DS'] / 'meta'
path_map['FTCNN_DS_CSV'] = path_map['FTCNN_DS_META'] / 'csv'
path_map['FTCNN_DS_SHP'] = path_map['FTCNN_DS_META'] / 'shp'

path_map['FTCNN_DS_MODEL'] = path_map['ROOT'] / 'datasets' / path_map['MODEL_NAME']
path_map['FTCNN_DS_MODEL_META'] = path_map['FTCNN_DS_MODEL'] / 'meta'
path_map['FTCNN_DS_MODEL_SHP'] = path_map['FTCNN_DS_MODEL_META'] / 'shp'
path_map['FTCNN_DS_MODEL_CSV'] = path_map['FTCNN_DS_MODEL_META'] / 'csv'

path_map['FTCNN_MODEL'] = path_map['FTCNN'] / path_map['MODEL_NAME']

path_map['FTCNN_DATA'] = path_map['FTCNN_MODEL'] / 'meta'
path_map['FTCNN_CONFIG_FILE'] = path_map['FTCNN_MODEL'] / 'config' / 'data.yaml'
path_map['FTCNN_YOLO_DATA_FILE'] = path_map['FTCNN_DATA'] / 'yolo_ndvi_ds.csv'

# Images and labels
path_map['FTCNN_IMAGES'] = path_map['FTCNN_MODEL'] / 'images'
path_map['FTCNN_LABELS'] = path_map['FTCNN_MODEL'] / 'labels'
path_map['FTCNN_LABELS_GENERATED'] = path_map['FTCNN_LABELS'] / 'generated'

path_map['FTCNN_CHIPS'] = path_map["FTCNN_IMAGES"] / 'chips'
path_map['FTCNN_PNGS'] = path_map["FTCNN_IMAGES"] / 'png'
path_map['FTCNN_TIFS'] = path_map["FTCNN_IMAGES"] / 'tif'

path_map['FTCNN_IMAGES_TRAIN'] = path_map['FTCNN_IMAGES'] / 'train'
path_map['FTCNN_IMAGES_TEST'] = path_map['FTCNN_IMAGES'] / 'test'
path_map['FTCNN_IMAGES_VAL'] = path_map['FTCNN_IMAGES'] / 'val'

path_map['FTCNN_LABELS_TRAIN'] = path_map['FTCNN_LABELS'] / 'train'
path_map['FTCNN_LABELS_TEST'] = path_map['FTCNN_LABELS'] / 'test'
path_map['FTCNN_LABELS_VAL'] = path_map['FTCNN_LABELS'] / 'val'

# Data
path_map['PRED_SHP'] = path_map['SHAPE_FILES'] / 'ModelPredictions'

# Zone 10
path_map['SHPZ10_SHP'] = path_map['SHAPE_FILES'] / 'Treatments_UTMz10_Only_08-18-24' / 'Treatments_UTMz10_Only_08-18-24.shp'
path_map['CSVZ10'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz10.csv'
path_map['CSVZ10_NORM'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz10_normalized.csv'
path_map['CSVZ10_CLEANED'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz10_normalized_cleaned.csv'
path_map['CSVZ10_CHIPPED'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz10_normalized_chipped.csv'
path_map['CSVZ10_CHIP_LABELS_UTM'] = path_map['FTCNN_DATA'] / 'Treatments_z10utm_chip_labels.csv'
path_map['CSVZ10_CHIP_LABELS_PIXEL'] = path_map['FTCNN_DATA'] / 'Treatments_z10pixel_chip_labels.csv'
path_map['CSVZ10_CHIP_LABELS_PIXEL_ENCODED'] = path_map['FTCNN_DATA'] / 'Treatments_z10pixel_chip_labels_encoded.csv'
path_map['CSVZ10_CHIP_LABELS_PREYOLO'] = path_map['FTCNN_DATA'] / 'Treatments_z10pixel_chip_labels_encoded_preyolo.csv'
path_map['SHPZ10_PRED_SHP'] = path_map['PRED_SHP'] / f"Treatmentsz10_{path_map['MODEL_NAME']}.shp"

# Zone 11
path_map['SHPZ11_SHP'] = path_map['SHAPE_FILES'] / 'Treatments_UTMz11_Only_08-18-24' / 'Treatments_UTMz11_Only_08-18-24.shp'
path_map['CSVZ11'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz11.csv'
path_map['CSVZ11_NORM'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz11_normalized.csv'
path_map['CSVZ11_CLEANED'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz11_normalized_cleaned.csv'
path_map['CSVZ11_CHIPPED'] = path_map['FTCNN_DATA'] / 'Treatments_UTMz11_normalized_chipped.csv'
path_map['CSVZ11_CHIP_LABELS_UTM'] = path_map['FTCNN_DATA'] / 'Treatments_z11utm_chip_labels.csv'
path_map['CSVZ11_CHIP_LABELS_PIXEL'] = path_map['FTCNN_DATA'] / 'Treatments_z11pixel_chip_labels.csv'
path_map['CSVZ11_CHIP_LABELS_PIXEL_ENCODED'] = path_map['FTCNN_DATA'] / 'Treatments_z11pixel_chip_labels_encoded.csv'
path_map['CSVZ11_CHIP_LABELS_PREYOLO'] = path_map['FTCNN_DATA'] / 'Treatments_z11pixel_chip_labels_encoded_preyolo.csv'
path_map['SHPZ11_PRED_SHP'] = path_map['PRED_SHP'] / f"Treatmentsz11_{path_map['MODEL_NAME']}.shp"


In [65]:
print("Creating directory structure")
for name, path in path_map.items():
    if isinstance(path, Path) and not path.suffix:
        path = path.resolve()
        path_map[name] = path
        path.mkdir(parents=True, exist_ok=True)
        print('  ',path)
print("Complete")

Creating directory structure
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/NDVI/NDVI Difference Rasters
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/Shapefiles
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/FTCNN_YOLO
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/meta
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/meta/csv
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/meta/shp
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/yolo_ftcnn_img-years=2022-2023_geom-years=ALL_chipsz=128_all
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/yolo_ftcnn_img-years=2022-2023_geom-years=ALL_chipsz=128_all/meta
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/yolo_ftcnn_img-years=2022-2023_geom-years=ALL_chipsz=128_all/meta/shp
   /home/joel/Dev/school/ssu-ai/ForestTreatment-CNN/datasets/yolo

In [66]:
# Make custom class parser (if required)
def classify(row):
    geom = row.get('geometry')
    return ("0", "Treatment") if geom is not None and not geom.is_empty and geom.area > 1 else ("-1", "Background")

In [68]:
shpz10 = ftcnn.load_shapefile(path_map['SHPZ10_SHP'])
print(shpz10.head())

In [69]:
# shpz10['StartYear'] = "2022"
# shpz10['EndYear'] = "2023"

# Filter Treatments by code
shpz10 = shpz10[shpz10['TreatmentT'] == "6"] 

# Fixes naming error in the original shape file
shpz10.loc[shpz10['Subregion'] == "Humboldt", "Subregion"] = 'Humboldt4'
shpz10.head()

In [70]:
ftcnn.save_as_csv(shpz10, path_map['FTCNN_DS_CSV'] / f'base_all_years={YEARS[0]}to{YEARS[1]}' / 'Treatments_UTMz10_Only_08-18-24.csv', exist_ok=True)
ftcnn.save_as_shp(shpz10, path_map['FTCNN_DS_SHP'] / f'base_all_years={YEARS[0]}to{YEARS[1]}'/ 'Treatments_UTMz10_Only_08-18-24.shp', exist_ok=True)

In [71]:
gdf = ftcnn.preprocess_shapefile(
    path_map['FTCNN_DS_SHP'] / 'base_all_years=2022to2023'/ 'Treatments_UTMz10_Only_08-18-24.shp',
    id_column="Subregion",
    img_dir=path_map['NDVI'],
)

In [72]:
ftcnn.save_as_csv(gdf, path_map['FTCNN_DS_CSV'] / f'preprocessed_all_years={YEARS[0]}to{YEARS[1]}'/ 'Treatments_UTMz10_Only_08-18-24_flattened.csv', exist_ok=True)
ftcnn.save_as_shp(gdf, path_map['FTCNN_DS_SHP'] / f'preprocessed_all_years={YEARS[0]}to{YEARS[1]}'/ 'Treatments_UTMz10_Only_08-18-24_flattened.shp', exist_ok=True)

In [73]:
# gdf, paths = ftcnn.make_ndvi_dataset(
#     path_map['FTCNN_DS_SHP'] / 'base_all_years=2022to2023'/ 'Treatments_UTMz10_Only_08-18-24.shp',
#     ndvi_dir=path_map['NDVI'],
#     output_dir=path_map['FTCNN_DS_MODEL'],
#     id_column="Subregion",
#     start_year_col="start_year",
#     end_year_col="end_year",
#     chip_size=CHIP_SIZE,
#     clean_dest=True,
#     xy_to_index=False,
#     exist_ok=True,
#     save_csv=True,
#     save_shp=True,
#     ignore_empty_geom=True,
#     tif_to_png=True,
#     leave=True,
# )

In [74]:
print(reload(ftcnn.ftcnn))
print(reload(ft))
print(reload(ftcnn.utils))
print(reload(ftcnn.modeling))
print(reload(ftcnn.modeling.yolo))


<module 'ftcnn.ftcnn' from '/home/joel/Dev/python/ftcnn/ftcnn.py'>
<module 'ftcnn.ftcnn' from '/home/joel/Dev/python/ftcnn/ftcnn.py'>
<module 'ftcnn.utils' from '/home/joel/Dev/python/ftcnn/utils.py'>
<module 'ftcnn.modeling' from '/home/joel/Dev/python/ftcnn/modeling/__init__.py'>
<module 'ftcnn.modeling.yolo' from '/home/joel/Dev/python/ftcnn/modeling/yolo.py'>


In [75]:
yolo_ds,_ = ft.ndvi_to_yolo_dataset(
    path_map['FTCNN_DS_SHP'] / 'base_all_years=2022to2023'/ 'Treatments_UTMz10_Only_08-18-24.shp',
    ndvi_dir=path_map['NDVI'],
    output_dir=path_map['FTCNN_DS_MODEL'],
    id_column="Subregion",
    start_year_col="start_year",
    end_year_col="end_year",
    chip_size=CHIP_SIZE,
    clean_dest=True,
    xy_to_index=True,
    exist_ok=True,
    save_csv=True,
    save_shp=True,
    ignore_empty_geom=True,
    generate_train_data=True,
    tif_to_png=True,
    split_mode=SPLIT_MODE,
    shuffle=False,
)

Creating NDVI dataset - Preprocessing shapefile:   0%|          | 0/6 [00:00<?, ?it/s]

Flattening geometry:   0%|          | 0/28 [00:00<?, ?it/s]

Cleaning output directory:   0%|          | 0/1 [00:00<?, ?it/s]

Creating GeoTIFF chips of size (128,128):   0%|          | 0/29 [00:00<?, ?it/s]

Processing ElDorado2_2022to2023_NDVI_Difference.tif:   0%|          | 0/35 [00:00<?, ?it/s]

...

Compiling YOLODataset labels and images:   0%|          | 0/197 [00:00<?, ?it/s]

Compiling YOLODataset labels and images:   0%|          | 0/197 [00:00<?, ?it/s]

Compiling YOLODataset labels and images:   0%|          | 0/197 [00:00<?, ?it/s]

Compiling YOLODataset labels and images:   0%|          | 0/197 [00:00<?, ?it/s]

Compiling YOLODataset labels and images:   0%|          | 0/2 [00:00<?, ?it/s]

In [76]:
yolo_ds.summary()

YOLO Dataset summary
Number of labels: 3154
Number of images: 2667
Number of classes: 1
Training data: 2000 images, 2357 labels
Validation data: 667 images, 797 labels
Class distribution:
{
  "Treatment": 3154
}

Data:
       type  class_id class_name    bbox_x    bbox_y    bbox_w    bbox_h  \
0     train         0  Treatment  0.695312  0.281250  0.296875  0.710938   
1     train         0  Treatment  0.000000  0.320312  0.562500  0.671875   
2     train         0  Treatment  0.000000  0.000000  0.054688  0.187500   
3     train         0  Treatment  0.398438  0.000000  0.117188  0.164062   
4     train         0  Treatment  0.000000  0.000000  0.359375  0.406250   
...     ...       ...        ...       ...       ...       ...       ...   
3149    val         0  Treatment  0.000000  0.234375  0.382812  0.757812   
3150    val         0  Treatment  0.617188  0.062500  0.375000  0.359375   
3151    val         0  Treatment  0.632812  0.492188  0.359375  0.500000   
3152    val         0